In [2]:
import pyspark.sql.functions
from pyspark.sql import SparkSession

In [16]:
spark = SparkSession \
    .builder \
    .appName("grab_dataframe") \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector:10.0.5') \
    .getOrCreate()

In [17]:
database = "hacking"
collection = "people"
connectionString = "mongodb://127.0.0.1"

In [18]:
df = spark.read.format("mongodb") \
    .option("database", database) \
    .option("connection.uri", connectionString) \
    .option("collection", collection) \
    .load()
print(df)

DataFrame[_id: string, age: int, name: string]


In [19]:
df.show()

+--------------------+----+-----+
|                 _id| age| name|
+--------------------+----+-----+
|636d401e920e9fad5...|  30| John|
|636d401e920e9fad5...|  45|  Joe|
|636d401e920e9fad5...|null|Jason|
+--------------------+----+-----+



### indexing [] gets column object

In [8]:
type(df["name"])

pyspark.sql.column.Column

### select method returns a dataframe of that column

In [9]:
type(df.select("name"))

df.select("name").show()

+-----+
| name|
+-----+
| John|
|  Joe|
|Jason|
+-----+



### head method will grab N Rows from the top of the dataframe

In [10]:
df.head(2)

[Row(_id='636d401e920e9fad564fb66b', age=30, name='John'),
 Row(_id='636d401e920e9fad564fb66c', age=45, name='Joe')]

In [11]:
row = df.head(3)[0]

In [39]:
df.select(["name", "age"]).show()

+-----+----+
| name| age|
+-----+----+
|  Joe|null|
| John|  39|
|Jason|  49|
+-----+----+



### withColumn will return a new dataframe with this new column added

In [12]:
df = df.withColumn("double_age", df["age"] * 2)
df.show()

+--------------------+----+-----+----------+
|                 _id| age| name|double_age|
+--------------------+----+-----+----------+
|636d401e920e9fad5...|  30| John|        60|
|636d401e920e9fad5...|  45|  Joe|        90|
|636d401e920e9fad5...|null|Jason|      null|
+--------------------+----+-----+----------+



### withColumnRenamed to rename a column

In [13]:
df.withColumnRenamed("age", "new_age")

DataFrame[_id: string, new_age: int, name: string, double_age: int]

### use SQL

In [14]:
df.createOrReplaceTempView("people")

spark.sql("select * from people").show()
spark.sql("select * from people where age > 40").show()
spark.sql("select * from people order by age desc").show()
spark.sql("select * from people order by age asc").show()
spark.sql("select * from people where age is not null").show()
spark.sql("select * from people where age is null").show()

age = 40
spark.sql("select * from people where age > {0}".format(age)).show()

+--------------------+----+-----+----------+
|                 _id| age| name|double_age|
+--------------------+----+-----+----------+
|636d401e920e9fad5...|  30| John|        60|
|636d401e920e9fad5...|  45|  Joe|        90|
|636d401e920e9fad5...|null|Jason|      null|
+--------------------+----+-----+----------+

+--------------------+---+----+----------+
|                 _id|age|name|double_age|
+--------------------+---+----+----------+
|636d401e920e9fad5...| 45| Joe|        90|
+--------------------+---+----+----------+

+--------------------+----+-----+----------+
|                 _id| age| name|double_age|
+--------------------+----+-----+----------+
|636d401e920e9fad5...|  45|  Joe|        90|
|636d401e920e9fad5...|  30| John|        60|
|636d401e920e9fad5...|null|Jason|      null|
+--------------------+----+-----+----------+

+--------------------+----+-----+----------+
|                 _id| age| name|double_age|
+--------------------+----+-----+----------+
|636d401e920e9fad

In [15]:
from pyspark.sql.functions import asc, desc

df.filter("age > 40").show()
df.filter("age is not null").show()
df.orderBy(desc("age")).show()
df.orderBy(asc("age")).show()


df.filter("age is not null and length(name) >= 4").orderBy(desc("age")).select("name").show()



+--------------------+---+----+----------+
|                 _id|age|name|double_age|
+--------------------+---+----+----------+
|636d401e920e9fad5...| 45| Joe|        90|
+--------------------+---+----+----------+

+--------------------+---+----+----------+
|                 _id|age|name|double_age|
+--------------------+---+----+----------+
|636d401e920e9fad5...| 30|John|        60|
|636d401e920e9fad5...| 45| Joe|        90|
+--------------------+---+----+----------+

+--------------------+----+-----+----------+
|                 _id| age| name|double_age|
+--------------------+----+-----+----------+
|636d401e920e9fad5...|  45|  Joe|        90|
|636d401e920e9fad5...|  30| John|        60|
|636d401e920e9fad5...|null|Jason|      null|
+--------------------+----+-----+----------+

+--------------------+----+-----+----------+
|                 _id| age| name|double_age|
+--------------------+----+-----+----------+
|636d401e920e9fad5...|null|Jason|      null|
|636d401e920e9fad5...|  30| J